# Analytics

#### Date: 2020/02

#### SUMMARY:

- This notebook represents the project quality analysis of the date exposed right above. 

### TEAM: SiGeD

##### Semester: 2020/02
##### Professor: Hilmer Neri

##### Members:

- Vitor Leal
- Gabriel Carvalho
- Ezequiel de Oliveira
- Gabriela Guedes
- Davi Marinho da Silva Campos
- João Pedro Alves da Silva Chaves
- Thiago França Vale Oliveira
- Lucas da Cunha Andrade
- Dafne Moretti Moreira
- Victor Yukio Cavalcanti Miki

### LIBRARIES

In [1]:
!pip3 install -r requirements.txt

In [2]:
# Deal with data
import pandas as pd
import numpy as np
import json

# Deal with API request
import urllib3
from urllib3 import request

# Deal with visualization
import seaborn as sns
import matplotlib.pyplot as plt

### GRAPH SETTINGS

In [3]:
%config InlineBackend.figure_format ='retina'
sns.set(font_scale=1.5)
sns.set_style('darkgrid',
              {'xtick.bottom' : True,
               'ytick.left': True,
               'grid.linestyle':'--',
               'font.monospace': ['Computer Modern Typewriter'],
               'axes.edgecolor' : 'white'})

### DATAFRAME SETTINGS

In [4]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

### SonarCloud

#### KEYS

In [5]:
front_key = 'fga-eps-mds_2020-2-G4-Frontend'

In [6]:
clients_key = 'fga-eps-mds_2020-2-G4-Clients'

In [7]:
demands_key = 'fga-eps-mds_2020-2-G4-Demands'

In [8]:
users_key = 'fga-eps-mds_2020-2-G4-Demands'

In [9]:
sectors_key = 'fga-eps-mds_2020-2-G4-Sectors'

#### METRICS

In [10]:
metric_list = ['files',
               'functions',
               'complexity',
               'comment_lines_density',
               'duplicated_lines_density',
               'security_rating',
               'tests',
               'test_success_density',
               'test_execution_time',
               'reliability_rating']

len(metric_list)

10

In [11]:
def generate_metric_string(metric_list):
    metric_str = ''
    
    for metric in metric_list:
        metric_str += metric + ','
        
    return metric_str

In [12]:
metric_str = generate_metric_string(metric_list)

#### URLS

In [13]:
front_url = f"https://sonarcloud.io/api/measures/component_tree?component={front_key}&metricKeys={metric_str}&ps=500"
clients_url = f"https://sonarcloud.io/api/measures/component_tree?component={clients_key}&metricKeys={metric_str}&ps=500"
demands_url = f"https://sonarcloud.io/api/measures/component_tree?component={demands_key}&metricKeys={metric_str}&ps=500"
users_url = f"https://sonarcloud.io/api/measures/component_tree?component={users_key}&metricKeys={metric_str}&ps=500"
sectors_url = f"https://sonarcloud.io/api/measures/component_tree?component={sectors_key}&metricKeys={metric_str}&ps=500"

#### API REQUEST

In [14]:
http = urllib3.PoolManager()

In [15]:
front_request = http.request('GET', front_url)
front_request.status

503

In [16]:
clients_request = http.request('GET', clients_url)
clients_request.status

503

In [17]:
demands_request = http.request('GET', demands_url)
demands_request.status

503

In [18]:
users_request = http.request('GET', users_url)
users_request.status

503

In [19]:
sectors_request = http.request('GET', sectors_url)
sectors_request.status

503

#### JSON DECODING

In [20]:
front_json = json.loads(front_request.data.decode('utf-8'))
clients_json = json.loads(clients_request.data.decode('utf-8'))
demands_json = json.loads(demands_request.data.decode('utf-8'))
users_json = json.loads(users_request.data.decode('utf-8'))
sectors_json = json.loads(sectors_request.data.decode('utf-8'))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## DATA

### PROJECT

In [ ]:
project_front_json = front_json['baseComponent']['measures']
project_clients_json = clients_json['baseComponent']['measures']
project_demands_json = demands_json['baseComponent']['measures']
project_users_json = users_json['baseComponent']['measures']
project_sectors_json = sectors_json['baseComponent']['measures']

In [ ]:
project_front_data = pd.DataFrame(project_front_json)
project_clients_data = pd.DataFrame(project_clients_json)
project_demands_data = pd.DataFrame(project_demands_json)
project_users_data = pd.DataFrame(project_users_json)
project_sectors_data = pd.DataFrame(project_sectors_json)

##### FRONT

In [ ]:
project_front_data

##### BACK

In [ ]:
project_clients_data

In [ ]:
project_demands_data

In [ ]:
project_users_data

In [ ]:
project_sectors_data

### FILES

In [ ]:
def metric_per_file(json):
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

In [ ]:
front_file_json = metric_per_file(front_json)
clients_file_json = metric_per_file(front_json)
demands_file_json = metric_per_file(front_json)
users_file_json = metric_per_file(front_json)
sectors_file_json = metric_per_file(front_json)

In [ ]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df = df.rename({'index': 'path'}, axis=1).drop(['files'], axis=1)

    return df

In [ ]:
front_files_data = generate_file_dataframe(metric_list, front_file_json, language_extension = 'js')
clients_files_data = generate_file_dataframe(metric_list, clients_file_json, language_extension = 'js')
demands_files_data = generate_file_dataframe(metric_list, demands_file_json, language_extension = 'js')
users_files_data = generate_file_dataframe(metric_list, users_file_json, language_extension = 'js')
sectors_files_data = generate_file_dataframe(metric_list, sectors_file_json, language_extension = 'js')

##### FRONT

In [ ]:
front_files_data

In [ ]:
clients_files_data

In [ ]:
demands_files_data

In [ ]:
users_files_data

In [ ]:
sectors_files_data

# ANALYSIS

## MAINTAINABILITY

### CODE QUALITY

##### COMPLEXITY

In [21]:
def m1(df):
    
    density_non_complex_files = round((len(df[(df['complexity'].astype(float)/df['functions'].astype(float)) < 10])/len(df))*100, 2)
    
    return density_non_complex_files

##### COMMENTS

In [22]:
def m2(df):
    
    density_comment_files = round((len(df[(df['comment_lines_density'].astype(float) > 10) & (df['comment_lines_density'].astype(float) < 30)])/len(df))*100, 2)
    
    return density_comment_files

##### DUPLICATIONS

In [23]:
def m3(df):
    
    duplication = round((len(df[(df['duplicated_lines_density'].astype(float) < 5)])/len(df))*100, 2)
    
    return duplication

### BLOCKING CODE

#### NON-BLOCKING FILES

In [24]:
def m4(df):
    
    non_blocking_files = round((len(df[(df['security_rating'].astype(float) >= 4)])/len(df))*100,2)
    
    return non_blocking_files

## RELIABILITY

#### TEST SUCCESS

In [ ]:
def m5(df):
    
    test_success_file = df[['path', 'test_success_density']]
    test_success_repository = df['test_success_density'].astype(float).mean()
    
    print("Project test unit density: ", test_success_repository)
    
    return test_success_file

#### FAST TESTS

In [ ]:
def m6(df):
    
    fast_test_df = df[(df['test_execution_time'].astype(float) < 300)]
    fast_test_df['fast_test'] = fast_test_df['test_execution_time']/fast_test_df['tests']
    
    fast_test_file = fast_test_df[['path', 'fast_test']]
    
    fast_test_repository = fast_test_df['fast_test'].astype(float).mean()
    
    print("Project test unit density: ", fast_test_repository)
    
    return fast_test_file

## PRODUCTIVITY

#### RESOLVED ISSUES' THROUGHPUT

In [25]:
def m7(number_of_issues_resolved, number_of_issues):
    
    resolved_issues_throughput = round((number_of_issues_resolved / number_of_issues) * 100, 2)
    
    return resolved_issues_throughput

#### ISSUE TYPE IN A TIMEFRAME

In [26]:
def density(issue, number_of_issues):
    issue_density = round((issue / number_of_issues) * 100, 2)
    return issue_density

In [27]:
def m8(tag_dict, number_of_issues, no_sprint):
    
    issue_densities = pd.DataFrame({
         "hotfix": [density(tag_dict["HOTFIX"], number_of_issues)],
        "docs": [density(tag_dict["DOCS"], number_of_issues)],
        "feature": [density(tag_dict["FEATURE"], number_of_issues)],
        "arq": [density(tag_dict["ARQ"], number_of_issues)],
        "devops": [density(tag_dict["DEVOPS"], number_of_issues)],
        "analytics": [density(tag_dict["ANALYTICS"], number_of_issues)],
        "us": [density(tag_dict["US"], number_of_issues)],
        "easy": [density(tag_dict["EASY"], number_of_issues)],
        "medium": [density(tag_dict["MEDIUM"], number_of_issues)],
        "hard": [density(tag_dict["HARD"], number_of_issues)],
        "eps": [density(tag_dict["EPS"], number_of_issues)],
        "mds": [density(tag_dict["MDS"], number_of_issues)],
        "no_sprint": no_sprint
    })
    
    return issue_densities

#### BUGS RATIO

In [28]:
def m9(tag_dict, number_of_issues):
    
    bugs_ratio = round(((tag_dict["DOCS"] + tag_dict["FEATURE"] + tag_dict["ARQ"] + tag_dict["DEVOPS"] + tag_dict["ANALYTICS"]) / number_of_issues) * 100, 2)
    
    return bugs_ratio

## TIMEFRAME: SPRINT 0

In [29]:
NUMBER_OF_ISSUES_RESOLVED = 7
NUMBER_OF_ISSUES = 7

TAGS = {
    "HOTFIX": 0,
    "DOCS": 0,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 0,
    "ANALYTICS": 0,
    "US": 0,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 7,
    "MDS": 4
}

In [30]:
s0_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [31]:
s0_m8 = m8(TAGS, NUMBER_OF_ISSUES, 0)

In [32]:
s0_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 1

In [33]:
NUMBER_OF_ISSUES_RESOLVED = 10
NUMBER_OF_ISSUES = 11

TAGS = {
    "HOTFIX": 0,
    "DOCS": 1,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 3,
    "ANALYTICS": 0,
    "US": 0,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 9,
    "MDS": 2
}

In [34]:
s1_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [35]:
s1_m8 = m8(TAGS, NUMBER_OF_ISSUES, 1)

In [36]:
s1_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 2

In [37]:
NUMBER_OF_ISSUES_RESOLVED = 9
NUMBER_OF_ISSUES = 11

TAGS = {
    "HOTFIX": 0,
    "DOCS": 2,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 3,
    "ANALYTICS": 0,
    "US": 1,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 7,
    "MDS": 3
}

In [38]:
s2_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [39]:
s2_m8 = m8(TAGS, NUMBER_OF_ISSUES, 2)

In [40]:
s2_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 3

In [41]:
NUMBER_OF_ISSUES_RESOLVED = 11
NUMBER_OF_ISSUES = 14

TAGS = {
    "HOTFIX": 0,
    "DOCS": 3,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 1,
    "ANALYTICS": 0,
    "US": 1,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 6,
    "MDS": 6
}

In [42]:
s3_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [43]:
s3_m8 = m8(TAGS, NUMBER_OF_ISSUES, 3)

In [44]:
s3_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 4

In [45]:
NUMBER_OF_ISSUES_RESOLVED = 14
NUMBER_OF_ISSUES = 14

TAGS = {
    "HOTFIX": 3,
    "DOCS": 5,
    "FEATURE": 0,
    "ARQ": 1,
    "DEVOPS": 2,
    "ANALYTICS": 0,
    "US": 4,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 9,
    "MDS": 5
}

In [46]:
s4_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [47]:
s4_m8 = m8(TAGS, NUMBER_OF_ISSUES, 4)

In [48]:
s4_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 5

In [49]:
NUMBER_OF_ISSUES_RESOLVED = 17
NUMBER_OF_ISSUES = 17

TAGS = {
    "HOTFIX": 4,
    "DOCS": 10,
    "FEATURE": 0,
    "ARQ": 1,
    "DEVOPS": 1,
    "ANALYTICS": 1,
    "US": 2,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 10,
    "MDS": 6
}

In [50]:
s5_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [51]:
s5_m8 = m8(TAGS, NUMBER_OF_ISSUES, 5)

In [52]:
s5_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 6

In [53]:
NUMBER_OF_ISSUES_RESOLVED = 25
NUMBER_OF_ISSUES = 25

TAGS = {
    "HOTFIX": 14,
    "DOCS": 6,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 1,
    "ANALYTICS": 0,
    "US": 0,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 11,
    "MDS": 17
}

In [54]:
s6_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [55]:
s6_m8 = m8(TAGS, NUMBER_OF_ISSUES, 6)

In [56]:
s6_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 7

In [57]:
NUMBER_OF_ISSUES_RESOLVED = 9
NUMBER_OF_ISSUES = 14

TAGS = {
    "HOTFIX": 0,
    "DOCS": 2,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 3,
    "ANALYTICS": 1,
    "US": 6,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 5,
    "MDS": 6
}

In [58]:
s7_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [59]:
s7_m8 = m8(TAGS, NUMBER_OF_ISSUES, 7)

In [60]:
s7_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 8

In [61]:
NUMBER_OF_ISSUES_RESOLVED = 9
NUMBER_OF_ISSUES = 11

TAGS = {
    "HOTFIX": 1,
    "DOCS": 3,
    "FEATURE": 0,
    "ARQ": 1,
    "DEVOPS": 1,
    "ANALYTICS": 0,
    "US": 5,
    "EASY": 0,
    "MEDIUM": 0,
    "HARD": 0,
    "EPS": 4,
    "MDS": 7
}

In [62]:
s8_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [63]:
s8_m8 = m8(TAGS, NUMBER_OF_ISSUES, 8)

In [64]:
s8_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 9

In [65]:
NUMBER_OF_ISSUES_RESOLVED = 16
NUMBER_OF_ISSUES = 16

TAGS = {
    "HOTFIX": 4,
    "DOCS": 3,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 2,
    "ANALYTICS": 1,
    "US": 7,
    "EASY": 10,
    "MEDIUM": 6,
    "HARD": 0,
    "EPS": 6,
    "MDS": 11
}

In [66]:
s9_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [67]:
s9_m8 = m8(TAGS, NUMBER_OF_ISSUES, 9)

In [68]:
s9_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 10

In [69]:
NUMBER_OF_ISSUES_RESOLVED = 20
NUMBER_OF_ISSUES = 20

TAGS = {
    "HOTFIX": 5,
    "DOCS": 6,
    "FEATURE": 0,
    "ARQ": 1,
    "DEVOPS": 3,
    "ANALYTICS": 0,
    "US": 8,
    "EASY": 13,
    "MEDIUM": 7,
    "HARD": 0,
    "EPS": 9,
    "MDS": 11
}

In [70]:
s10_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [71]:
s10_m8 = m8(TAGS, NUMBER_OF_ISSUES, 10)

In [72]:
s10_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 11

In [73]:
NUMBER_OF_ISSUES_RESOLVED = 14
NUMBER_OF_ISSUES = 14

TAGS = {
    "HOTFIX": 0,
    "DOCS": 4,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 0,
    "ANALYTICS": 0,
    "US": 4,
    "EASY": 4,
    "MEDIUM": 7,
    "HARD": 0,
    "EPS": 6,
    "MDS": 7
}

In [74]:
s11_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [75]:
s11_m8 = m8(TAGS, NUMBER_OF_ISSUES, 11)

In [76]:
s11_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 12

In [77]:
NUMBER_OF_ISSUES_RESOLVED = 1
NUMBER_OF_ISSUES = 2

TAGS = {
    "HOTFIX": 0,
    "DOCS": 0,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 1,
    "ANALYTICS": 1,
    "US": 0,
    "EASY": 0,
    "MEDIUM": 1,
    "HARD": 0,
    "EPS": 1,
    "MDS": 0
}

In [78]:
s12_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [79]:
s12_m8 = m8(TAGS, NUMBER_OF_ISSUES, 12)

In [80]:
s12_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 13

In [81]:
NUMBER_OF_ISSUES_RESOLVED = 15
NUMBER_OF_ISSUES = 15

TAGS = {
    "HOTFIX": 0,
    "DOCS": 2,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 1,
    "ANALYTICS": 1,
    "US": 10,
    "EASY": 7,
    "MEDIUM": 4,
    "HARD": 1,
    "EPS": 2,
    "MDS": 12
}

In [82]:
s13_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [83]:
s13_m8 = m8(TAGS, NUMBER_OF_ISSUES, 13)

In [84]:
s13_m9 = m9(TAGS, NUMBER_OF_ISSUES)

## TIMEFRAME: SPRINT 14

In [85]:
NUMBER_OF_ISSUES_RESOLVED = 0
NUMBER_OF_ISSUES = 11

TAGS = {
    "HOTFIX": 1,
    "DOCS": 6,
    "FEATURE": 0,
    "ARQ": 0,
    "DEVOPS": 1,
    "ANALYTICS": 2,
    "US": 0,
    "EASY": 3,
    "MEDIUM": 6,
    "HARD": 0,
    "EPS": 7,
    "MDS": 5
}

In [86]:
s14_m7 = m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

In [87]:
s14_m8 = m8(TAGS, NUMBER_OF_ISSUES, 14)

In [88]:
s14_m9 = m9(TAGS, NUMBER_OF_ISSUES)

### METRIC RESULTS

In [89]:
def create_metrics_df(m7_list, m9_list, dates):
    
# data_inicio   data_fim    m7   m9    asc2    totalAC2    no_sprint
    start_date = []
    end_date = []
    asc2_list = []
    total_asc2_list = []
    
    for sprint_dates in dates:
        start_date.append(sprint_dates[0])
        end_date.append(sprint_dates[1])
        
    for i in range(15):
        asc2 = (m7_list[i] * 0.5) + (m9_list[i] * 0.5)
        asc2_list.append(asc2)
        total_asc2_list.append(asc2 * 0.5)
        
        

        
    metrics_df = pd.DataFrame({'data_inicio': start_date,
                               'data_fim': end_date,
                               'm7': m7_list,
                               'm9': m9_list, 
                               'asc2': asc2_list,
                               'totalAC2': total_asc2_list,
                               'no_sprint': range(15)
                              })
        
    return metrics_df

In [90]:
metrics_m7 = [s0_m7, s1_m7, s2_m7, s3_m7, s4_m7, s5_m7, s6_m7, s7_m7, s8_m7, s9_m7, s10_m7, s11_m7, s12_m7, s13_m7, s14_m7]
metrics_m9 = [s0_m9, s1_m9, s2_m9, s3_m9, s4_m9, s5_m9, s6_m9, s7_m9, s8_m9, s9_m9, s10_m9, s11_m9, s12_m9, s13_m9, s14_m9]

dates = [
    ('02/02/2021', '12/02/2021'),
    ('13/02/2021', '19/02/2021'),
    ('20/02/2021', '26/02/2021'),
    ('27/02/2021', '05/03/2021'),
    ('05/03/2021', '12/03/2021'),
    ('12/03/2021', '19/03/2021'),
    ('19/03/2021', '26/03/2021'),
    ('26/03/2021', '02/04/2021'),
    ('02/04/2021', '09/04/2021'),
    ('09/04/2021', '16/04/2021'),
    ('16/04/2021', '23/04/2021'),
    ('23/04/2021', '30/04/2021'),
    ('30/04/2021', '07/05/2021'),
    ('07/05/2021', '14/05/2021'),
    ('14/05/2021', '21/05/2021')
        ]
        
metrics_df = create_metrics_df(metrics_m7, metrics_m9, dates)

In [91]:
metrics_df

,data_inicio,data_fim,m7,m9,asc2,totalAC2,no_sprint
0,02/02/2021,12/02/2021,100.00,0.00,50.000,25.0000,0
1,13/02/2021,19/02/2021,90.91,36.36,63.635,31.8175,1
2,20/02/2021,26/02/2021,81.82,45.45,63.635,31.8175,2
3,27/02/2021,05/03/2021,78.57,28.57,53.570,26.7850,3
4,05/03/2021,12/03/2021,100.00,57.14,78.570,39.2850,4
5,12/03/2021,19/03/2021,100.00,76.47,88.235,44.1175,5
6,19/03/2021,26/03/2021,100.00,28.00,64.000,32.0000,6
7,26/03/2021,02/04/2021,64.29,42.86,53.575,26.7875,7
8,02/04/2021,09/04/2021,81.82,45.45,63.635,31.8175,8
9,09/04/2021,16/04/2021,100.00,37.50,68.750,34.3750,9


In [92]:
metrics_df.to_csv('data/fga-eps-mds-2020-2-SiGeD-qualidade-total-processo-SPRINT.csv', index = False)

In [93]:
m8_metrics = pd.concat([s0_m8, s1_m8, s2_m8, s3_m8, s4_m8, s5_m8, s6_m8, s7_m8, s8_m8, s9_m8, s10_m8, s11_m8, s12_m8, s13_m8, s14_m8], ignore_index=True)


In [94]:
m8_metrics

,hotfix,docs,feature,arq,devops,analytics,us,easy,medium,hard,eps,mds,no_sprint
0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,57.14,0
1,0.00,9.09,0.0,0.00,27.27,0.00,0.00,0.00,0.00,0.00,81.82,18.18,1
2,0.00,18.18,0.0,0.00,27.27,0.00,9.09,0.00,0.00,0.00,63.64,27.27,2
3,0.00,21.43,0.0,0.00,7.14,0.00,7.14,0.00,0.00,0.00,42.86,42.86,3
4,21.43,35.71,0.0,7.14,14.29,0.00,28.57,0.00,0.00,0.00,64.29,35.71,4
5,23.53,58.82,0.0,5.88,5.88,5.88,11.76,0.00,0.00,0.00,58.82,35.29,5
6,56.00,24.00,0.0,0.00,4.00,0.00,0.00,0.00,0.00,0.00,44.00,68.00,6
7,0.00,14.29,0.0,0.00,21.43,7.14,42.86,0.00,0.00,0.00,35.71,42.86,7
8,9.09,27.27,0.0,9.09,9.09,0.00,45.45,0.00,0.00,0.00,36.36,63.64,8
9,25.00,18.75,0.0,0.00,12.50,6.25,43.75,62.50,37.50,0.00,37.50,68.75,9


In [95]:
metrics_df.to_csv('data/fga-eps-mds-2020-2-SiGeD-qualidade-total-processo-m8-SPRINT.csv', index = False)